# Big Ten (B1G) Football Schedule Model

This notebook contains a mixed integer programming model (MIP) for creating a football schedule for the B1G Conference.

# Information

* The B1G Conference has two 7 team divisions
    * West Division
    * East Division
* There are 13 weeks in the season. Each team has 
    * A bye week
    * 3 Non-conference games
    * 9 Conference games
        * 6 In-division Games (East team plays East team, West team plays West team)
        * 3 Crossover games (a West team playing an East team)
    * 4 or 5 home conference games
    * 3 home in-division games

# Import Packages

In [1]:
import pulp
import pandas as pd

# Read In the Data

In [2]:
team_data = pd.read_csv(r"data\teams.csv")

In [3]:
# remove spaces in team name
team_data.Team = [x.replace(" ","") for x in team_data.Team]

# Sets

* I: set of all B1G teams
* W: set of teams in the West Division (a subset of I)
* E: set of teams in the East Division (a subset of I)
* T: set of weeks

In [4]:
# teams
I = team_data.Team.tolist()
W = team_data.loc[team_data.Division == "West",['Team']].Team.tolist()
E = team_data.loc[team_data.Division == "East",['Team']].Team.tolist()

# weeks
T = range(1,14)

# Decision Variables

* $ x_{hat}$: A binary variable that equals 1 if team $h \in I$ is home against team $a \in I, a \neq h$, during week $ t \in T$ and 0 otherwise
* $ y_{it}$: A binary variable that equals 1 if team $i \in I$ plays a non-conference game during week $ t \in T$ and 0 otherwise
* $ z_{it}$: A binary variable that equals 1 if team $i \in I$ has a bye in week $ t \in T$ and 0 otherwise

In [5]:
x = pulp.LpVariable.dicts('x',(I,I,T),cat=pulp.LpBinary)
y = pulp.LpVariable.dicts('y',(I,T),cat=pulp.LpBinary)
z = pulp.LpVariable.dicts('z',(I,T),cat=pulp.LpBinary) 

The previous cell created extra decision variables that we don't want. These are $ x_{hat}$ variables in which $h=a$. Because a team cannot play against itself, we need to remove them.

In [6]:
for t in T:
    
    for h in I:
        
        for a in I:
            
            if h == a:
                
                del x[h][a][t]

# Create the Problem in PuLP

In [7]:
schedule = pulp.LpProblem("Schedule",pulp.LpMinimize)

# Create the Objective Function

We are only concerned about creating a schedule that satisfies all of the constraints. We are not optimizing anything. Thus, we can simply set the objective function to 0.

In [8]:
schedule += 0

# Constraints

## Each team plays exactly 3 In-Division home games:

### West

$$ \sum_{t \in T} \sum_{a \in W,a \neq h} x_{hat} = 3, \qquad \forall h \in W $$

In [9]:
for h in W:
    schedule += pulp.lpSum(x[h][a][t] for t in T for a in W if a != h) == 3

### East

$$ \sum_{t \in T} \sum_{a \in E,a \neq h} x_{hat} = 3, \qquad \forall h \in E $$

In [10]:
for h in E:
    schedule += pulp.lpSum(x[h][a][t] for t in T for a in E if a != h) == 3

## Each team plays exactly 3 In-Division road games:

### West

$$ \sum_{t \in T} \sum_{h \in W,h \neq a} x_{hat} = 3, \qquad \forall a \in W $$

In [11]:
for a in W:
    schedule += pulp.lpSum(x[h][a][t] for t in T for h in W if h != a) == 3

### East

$$ \sum_{t \in T} \sum_{h \in E,h \neq a} x_{hat} = 3, \qquad \forall a \in E $$

In [12]:
for a in E:
    schedule += pulp.lpSum(x[h][a][t] for t in T for h in E if h != a) == 3

## Each team has either 4 or 5 conference home games

$$4 \leq \sum_{t \in T} \sum_{a \in I,h \neq i} x_{hat} \leq 5, \qquad \forall i \in I $$

In [13]:
for h in I:
    schedule += pulp.lpSum(x[h][a][t] for t in T for a in I if a != h) >= 4
    schedule += pulp.lpSum(x[h][a][t] for t in T for a in I if a != h) <= 5

## Each team has 3 crossover games

### West

$$ \sum_{t \in T} \sum_{a \in E} x_{iat} + \sum_{t \in T} \sum_{h \in E} x_{hit} = 3, \qquad \forall i \in W $$

In [14]:
for i in W:
    schedule += pulp.lpSum(x[i][a][t] for t in T for a in E) + pulp.lpSum(x[h][i][t] for t in T for h in E) == 3

### East

$$ \sum_{t \in T} \sum_{a \in W} x_{iat} + \sum_{t \in T} \sum_{h \in W} x_{hit} = 3, \qquad \forall i \in E $$

In [15]:
for i in E:
    schedule += pulp.lpSum(x[i][a][t] for t in T for a in W) + pulp.lpSum(x[h][i][t] for t in T for h in W) == 3

## Non-conference Schedule

$$ \sum_{t \in T}y_{it} = 3, \qquad \forall i \in I $$

In [16]:
for i in I:
    schedule += pulp.lpSum(y[i][t] for t in T) == 3

## Bye Week for each team

$$ \sum_{t = 3}^9 z_{it} = 1, \qquad \forall i \in I $$

In [17]:
for i in I:
    schedule += pulp.lpSum(z[i][t] for t in T[3:10]) == 1 # bye week is between weeks 3 and 9

## Each team has a game (or bye) for each week

$$ \sum_{a \in I}x_{iat} + \sum_{h \in I}x_{hit} + y_{it} + z_{it} = 1, \qquad \forall i \in I, t\in T $$

In [18]:
for t in T:
    for i in I:
        schedule += pulp.lpSum(x[i][a][t] for a in I if a != i) + pulp.lpSum(x[h][i][t] for h in I if h != i) +  y[i][t] + z[i][t] == 1

## Teams can only face off against each other at most once

$$ \sum_{t \in T}x_{ijt} + \sum_{t \in T}x_{jit} \leq 1, \qquad \forall i \in I, j \in I, j \neq i$$

In [19]:
for i in I:
    for j in I:
        if i != j:
            schedule += pulp.lpSum(x[i][j][t] for t in T) + pulp.lpSum(x[j][i][t] for t in T) <= 1 

# Solve the Problem

In [20]:
schedule.solve()

1

In [21]:
for t in T:
    print(f"Week: {t}")
    for h in I:
        for a in I:
            if a != h:
                if x[h][a][t].value() == 1.0:
                    home, away, week = x[h][a][t].getName().split('_')[1:]
                    print(f'B1G Match: {away} at {home}')
                    
    for i in I:
        if y[i][t].value() == 1.0:
            b1g_team, week = y[i][t].getName().split('_')[1:]
            print(f'Non-Conf Game: {b1g_team}')
    
    for i in I:
        if z[i][t].value() == 1.0:
            print(f"Bye for {i}")
            
    print()  # newline

Week: 1
B1G Match: Indiana at OhioState
B1G Match: Michigan at PennState
B1G Match: Northwestern at Iowa
B1G Match: Minnesota at Purdue
B1G Match: MichiganState at Wisconsin
Non-Conf Game: Maryland
Non-Conf Game: Rutgers
Non-Conf Game: Illinois
Non-Conf Game: Nebraska

Week: 2
B1G Match: Nebraska at PennState
B1G Match: MichiganState at Indiana
B1G Match: Rutgers at Purdue
B1G Match: Maryland at Wisconsin
B1G Match: OhioState at Northwestern
Non-Conf Game: Michigan
Non-Conf Game: Iowa
Non-Conf Game: Minnesota
Non-Conf Game: Illinois

Week: 3
B1G Match: Minnesota at MichiganState
B1G Match: Northwestern at Maryland
B1G Match: Michigan at Rutgers
B1G Match: PennState at Iowa
B1G Match: Purdue at Illinois
Non-Conf Game: OhioState
Non-Conf Game: Indiana
Non-Conf Game: Wisconsin
Non-Conf Game: Nebraska

Week: 4
B1G Match: Indiana at Michigan
B1G Match: PennState at Rutgers
B1G Match: Iowa at Illinois
B1G Match: Maryland at Nebraska
Non-Conf Game: Minnesota
Non-Conf Game: Northwestern
Bye fo

# Next Steps

* Create a nice output file for the complete schedule
* Create a dashboard
* Make program that allows for additional constraints
    * Michigan plays against Ohio State in week 13